In [ ]:
!git clone https://github.com/simoneliasen/query-selector.git
!pip3 install deepspeed
%cd query-selector

Cloning into 'query-selector'...
remote: Enumerating objects: 402, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 402 (delta 232), reused 272 (delta 132), pack-reused 18
Receiving objects: 100% (402/402), 11.79 MiB | 10.83 MiB/s, done.
Resolving deltas: 100% (236/236), done.
Checking out files: 100% (19/19), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 709 kB 25.0 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 145 kB 75.4 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.7.6-py3-none-any.whl size=714426 sha256=4890da0448bd09346994b4dfd1f6f2f4010b19f6a15e1b96082db67cea5cd2de
  Stored in directory: /root/.cache/pip/wheels/a8/8e/b2/e0eea301fe581983fcb99c6d25ee85b522f82390b0dc071157
Successfully built deepspeed
/content/query-selector


In [ ]:
#Get data
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
!pip install wandb -qU
import wandb

#Paper Dataset
ETTh1 = pd.read_csv("data/ETTh1.csv")

#Our Full Dataset
#dataset_dropNA = pd.read_csv("data/dataset_dropNA.csv") #old dataset without timefeatures
dataset_dropNA = pd.read_csv("data/datasetV3.csv")
dataset_dropNA.rename(columns = {'hour':'date'}, inplace = True) #model only works with date somehow

#Subset of Our Dataset (7 features, default for troubleshooting)
#Change sys.argv.extend(["--input_len", '372', "--output_len", "3"], with 372-372 or 372-1)
dataset_dropNA_subset = dataset_dropNA[['date', 'TH_NP15_GEN-APND', 'TH_SP15_GEN-APND','TH_ZP26_GEN-APND', 'CAISO-SP15 Wind Power Generation Forecast','CAISO-NEVP Power Demand Forecast','CAISO-AZPS Power Demand Forecast','CAISO-SDGE Power Demand Forecast']].copy()
dataset_dropNA_subset.to_csv('data/dataset_dropNA_subset.csv', index=False) 



def timestep_check(df_features, endate):
    xx = True
    try:
        df_features.index[df_features['date'] == str(endate)][0]
    except:
        xx = False

    return xx


def get_data(dates, traininglength, df_features):
    #to get date as datetime object, so timedelta method can be used.
    start_date = datetime.fromisoformat(dates)
    #timedelta finds automatic the date x days before 
    endindex = df_features.index[df_features['date'] == dates][0]
    
    i = 7
    timestep_exist = False
    #til at håndtere manglende timesteps i csv
    while timestep_exist is False:
        endate = start_date - timedelta(days= (traininglength+i))
        timestep_exist = timestep_check(df_features, endate)
        i += 1
        
    startindex = df_features.index[df_features['date'] == str(endate)][0]
 
    df_season = df_features.iloc[startindex:endindex]
   
    return df_season
 

def offset_dataframe(df,i) -> pd.DataFrame:
    later_validation_hours = (6-i)*24
    unused_train_hours = i*24
    df_copy = df.drop(df.tail(later_validation_hours).index) # drop last n rows
    df_copy = df_copy.drop(df_copy.head(unused_train_hours).index) # drop first n rows
    return df_copy


def split_dataframes(df, season_start):
    df.to_csv('data/season ' + str(season_start) + '.csv', index=False) #save to training.csv file
    for i in range(7):
      df_subset = offset_dataframe(df,i)
      df_subset.to_csv('data/training ' + str(i+1) + ' ' + str(season_start) + '.csv', index=False) #save as training,1,2,3,4... files .csv
      


     |████████████████████████████████| 1.9 MB 14.7 MB/s 
     |████████████████████████████████| 182 kB 49.9 MB/s 
     |████████████████████████████████| 168 kB 64.5 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 73.6 MB/s 
     |████████████████████████████████| 166 kB 64.9 MB/s 
     |████████████████████████████████| 166 kB 79.0 MB/s 
     |████████████████████████████████| 162 kB 78.9 MB/s 
     |████████████████████████████████| 162 kB 65.1 MB/s 
     |████████████████████████████████| 158 kB 68.2 MB/s 
     |████████████████████████████████| 157 kB 72.4 MB/s 
     |████████████████████████████████| 157 kB 81.0 MB/s 
     |████████████████████████████████| 157 kB 84.3 MB/s 
     |████████████████████████████████| 157 kB 81.3 MB/s 
     |████████████████████████████████| 157 kB 69.9 MB/s 
     |████████████████████████████████| 157 kB 82.7 MB/s 
     |████████████████████████████████| 157 kB 67.1 MB/s 
     |██████████

In [ ]:
#Parse json
def get_results():

    with open('data/datasave.json', 'r') as f:
      data = json.load(f)

    season1 = "s1"
    season2 = "s2"
    season3 = "s3"
    season4 = "s4"

    season1_RMSE = []
    season1_MAE = []
    season1_pred_NP15 = []  
    season1_pred_SP15 = []
    season1_pred_ZP26 = []
    season1_truth_NP15 = []
    season1_truth_SP15 = []
    season1_truth_ZP26 = []

    season2_RMSE = []
    season2_MAE = []
    season2_pred_NP15 = []  
    season2_pred_SP15 = []
    season2_pred_ZP26 = []
    season2_truth_NP15 = []
    season2_truth_SP15 = []
    season2_truth_ZP26 = []


    season3_RMSE = []
    season3_MAE = []
    season3_pred_NP15 = []  
    season3_pred_SP15 = []
    season3_pred_ZP26 = []
    season3_truth_NP15 = []
    season3_truth_SP15 = []
    season3_truth_ZP26 = []

    season4_RMSE = []
    season4_MAE = []
    season4_pred_NP15 = []  
    season4_pred_SP15 = []
    season4_pred_ZP26 = []
    season4_truth_NP15 = []
    season4_truth_SP15 = []
    season4_truth_ZP26 = []


    for key, value in data.items():
      if season1 in key: 
        season1_RMSE.append(value['RMSE'])
        season1_MAE.append(value['MAE'])

        for x in value['Predictions']: 
          season1_pred_NP15.append(x[0])
          season1_pred_SP15.append(x[1])
          season1_pred_ZP26.append(x[2])

        for x in value['Ground Truth']:
          season1_truth_NP15.append(x[0])
          season1_truth_SP15.append(x[1])
          season1_truth_ZP26.append(x[2])
    
      if season2 in key:
        season2_RMSE.append(value['RMSE'])
        season2_MAE.append(value['MAE'])

        for x in value['Predictions']: 
          season2_pred_NP15.append(x[0])
          season2_pred_SP15.append(x[1])
          season2_pred_ZP26.append(x[2])

        for x in value['Ground Truth']:
          season2_truth_NP15.append(x[0])
          season2_truth_SP15.append(x[1])
          season2_truth_ZP26.append(x[2])

      if season3 in key:
        season3_RMSE.append(value['RMSE'])
        season3_MAE.append(value['MAE'])

        for x in value['Predictions']: 
          season3_pred_NP15.append(x[0])
          season3_pred_SP15.append(x[1])
          season3_pred_ZP26.append(x[2])

        for x in value['Ground Truth']:
          season3_truth_NP15.append(x[0])
          season3_truth_SP15.append(x[1])
          season3_truth_ZP26.append(x[2])

      if season4 in key:
        season4_RMSE.append(value['RMSE'])
        season4_MAE.append(value['MAE'])

        for x in value['Predictions']: 
          season4_pred_NP15.append(x[0])
          season4_pred_SP15.append(x[1])
          season4_pred_ZP26.append(x[2])

        for x in value['Ground Truth']:
          season4_truth_NP15.append(x[0])
          season4_truth_SP15.append(x[1])
          season4_truth_ZP26.append(x[2])


    print("\n")
    print("Season 1")
    print("RMSE")
    print(season1_RMSE)
    print("MAE")
    print(season1_MAE)
    print("RMSE_average_season1")
    RMSE_average1 = sum(season1_RMSE) / len(season1_RMSE)
    print(RMSE_average1)
    print("MAE_average_season1")
    MAE_average1 = sum(season1_MAE) / len(season1_MAE)
    print(MAE_average1)
    print("NP15 pred")
    print(season1_pred_NP15)
    print("SP15 pred")
    print(season1_pred_SP15)
    print("ZP26 pred")
    print(season1_pred_ZP26)
    print("NP15 truth")
    print(season1_truth_NP15)
    print("SP15 truth")
    print(season1_truth_SP15)
    print("ZP26 truth")
    print(season1_truth_ZP26)


    print("\n")
    print("Season 2")
    print("RMSE")
    print(season2_RMSE)
    print("MAE")
    print(season2_MAE)
    print("RMSE_average_season2")
    RMSE_average2 = sum(season2_RMSE) / len(season2_RMSE)
    print(RMSE_average2)
    print("MAE_average_season2")
    MAE_average2 = sum(season2_MAE) / len(season2_MAE)
    print(MAE_average2)
    print("NP15 pred")
    print(season2_pred_NP15)
    print("SP15 pred")
    print(season2_pred_SP15)
    print("ZP26 pred")
    print(season2_pred_ZP26)
    print("NP15 truth")
    print(season2_truth_NP15)
    print("SP15 truth")
    print(season2_truth_SP15)
    print("ZP26 truth")
    print(season2_truth_ZP26)


    print("\n")
    print("Season 3")
    print("RMSE")
    print(season3_RMSE)
    print("MAE")
    print(season3_MAE)
    print("RMSE_average_season3")
    RMSE_average3 = sum(season3_RMSE) / len(season3_RMSE)
    print(RMSE_average3)
    print("MAE_average_season3")
    MAE_average3 = sum(season3_MAE) / len(season3_MAE)
    print(MAE_average3)
    print("NP15 pred")
    print(season3_pred_NP15)
    print("SP15 pred")
    print(season3_pred_SP15)
    print("ZP26 pred")
    print(season3_pred_ZP26)
    print("NP15 truth")
    print(season3_truth_NP15)
    print("SP15 truth")
    print(season3_truth_SP15)
    print("ZP26 truth")
    print(season3_truth_ZP26)


    print("\n")
    print("Season 4")
    print("RMSE")
    print(season4_RMSE)
    print("MAE")
    print(season4_MAE)
    print("RMSE_average_season4")
    RMSE_average4 = sum(season4_RMSE) / len(season4_RMSE)
    print(RMSE_average4)
    print("MAE_average_season3")
    MAE_average4 = sum(season4_MAE) / len(season4_MAE)
    print(MAE_average4)
    print("NP15 pred")
    print(season4_pred_NP15)
    print("SP15 pred")
    print(season4_pred_SP15)
    print("ZP26 pred")
    print(season4_pred_ZP26)
    print("NP15 truth")
    print(season4_truth_NP15)
    print("SP15 truth")
    print(season4_truth_SP15)
    print("ZP26 truth")
    print(season4_truth_ZP26)


    RMSE_seasons_average = []
    RMSE_seasons_average.append(RMSE_average1) 
    RMSE_seasons_average.append(RMSE_average2)
    RMSE_seasons_average.append(RMSE_average3)
    RMSE_seasons_average.append(RMSE_average4)

    MAE_seasons_average = []
    MAE_seasons_average.append(MAE_average1)
    MAE_seasons_average.append(MAE_average2)
    MAE_seasons_average.append(MAE_average3)
    MAE_seasons_average.append(MAE_average4)


    print("\n")
    print("RMSE Averages of seasons")
    print(RMSE_seasons_average)

    print("RMSE Total seasons average")
    Total_average_rmse_loss = sum(RMSE_seasons_average) / len(RMSE_seasons_average)
    print(Total_average_rmse_loss)

    print("\n")
    print("MAE Averages of seasons")
    print(MAE_seasons_average)

    print("MAE Total seasons average")
    Total_average_mae_loss = sum(MAE_seasons_average) / len(MAE_seasons_average)
    print(Total_average_mae_loss)

    #Predictions/targets for individual seasons
    for i in range(len(season1_pred_NP15)): #Any predicition or target series can be used all are 273
      wandb.log({"Winter predictions (39h) Hub: NP15": season1_pred_NP15[i],
                 "Winter predictions (39h) Hub: SP15": season1_pred_SP15[i],
                 "Winter predictions (39h) Hub: ZP26": season1_pred_ZP26[i],
                 "Spring predictions (39h) Hub: NP15": season2_pred_NP15[i],
                 "Spring predictions (39h) Hub: SP15": season2_pred_SP15[i],
                 "Spring predictions (39h) Hub: ZP26": season2_pred_ZP26[i],
                 "Summer predictions (39h) Hub: NP15": season3_pred_NP15[i],
                 "Summer predictions (39h) Hub: SP15": season3_pred_SP15[i],
                 "Summer predictions (39h) Hub: ZP26": season3_pred_ZP26[i],
                 "Fall predictions (39h) Hub: NP15": season4_pred_NP15[i],
                 "Fall predictions (39h) Hub: SP15": season4_pred_SP15[i],
                 "Fall predictions (39h) Hub: ZP26": season4_pred_ZP26[i],
                 "Winter targets (39h) Hub: NP15": season1_truth_NP15[i],
                 "Winter targets (39h) Hub: SP15": season1_truth_SP15[i],
                 "Winter targets (39h) Hub: ZP26": season1_truth_ZP26[i],
                 "Spring targets (39h) Hub: NP15": season2_truth_NP15[i],
                 "Spring targets (39h) Hub: SP15": season2_truth_SP15[i],
                 "Spring targets (39h) Hub: ZP26": season2_truth_ZP26[i],
                 "Summer targets (39h) Hub: NP15": season3_truth_NP15[i],
                 "Summer targets (39h) Hub: SP15": season3_truth_SP15[i],
                 "Summer targets (39h) Hub: ZP26": season3_truth_ZP26[i],
                 "Fall targets (39h) Hub: NP15": season4_truth_NP15[i],
                 "Fall targets (39h) Hub: SP15": season4_truth_SP15[i],
                 "Fall targets (39h) Hub: ZP26": season4_truth_ZP26[i]
                 })
      
    #7 values of RMSE/MAE over season
    for i in range(len(season1_RMSE)): #Any season series can be used all are 7
      wandb.log({"Winter_RMSE_loss": season1_RMSE[i],
                 "Spring_RMSE_loss": season2_RMSE[i],
                 "Summer_RMSE_loss": season3_RMSE[i],
                 "Fall_RMSE_loss": season4_RMSE[i],
                 "Winter_MAE_loss": season1_MAE[i],
                 "Spring_MAE_loss": season2_MAE[i],
                 "Summer_MAE_loss": season3_MAE[i],
                 "Fall_MAE_loss": season4_MAE[i]
                 })

    #Individual seasons averages
    wandb.log({"Winter_Average_MAE_Loss": MAE_average1})
    wandb.log({"Spring_Average_MAE_Loss": MAE_average2})
    wandb.log({"Summer_Average_MAE_Loss": MAE_average3})
    wandb.log({"Fall_Average_MAE_Loss": MAE_average4})

    wandb.log({"Winter_Average_RMSE_Loss": RMSE_average1})
    wandb.log({"Spring_Average_RMSE_Loss": RMSE_average2})
    wandb.log({"Summer_Average_RMSE_Loss": RMSE_average3})
    wandb.log({"Fall_Average_RMSE_Loss": RMSE_average4})

    #Logging Total Average of MAE and RMSE
    wandb.log({"Total_Average_MAE_Loss": Total_average_mae_loss})
    wandb.log({"Total_Average_RMSE_Loss": Total_average_rmse_loss})


#from matplotlib import pyplot as plt
#plt.plot(season1_pred_NP15)
#plt.plot(season1_truth_NP15)


In [ ]:
import os
os.environ["WANDB_AGENT_MAX_INITIAL_FAILURES"]= "100"

sweep_config = {
    'name': 'navn',
    'method': 'random', #grid, random, bayesian
    'metric': {
    'name': 'Total_Average_MAE_Loss',
    'goal': 'minimize'  
        },
    'parameters': {
        'batch_size': {
            'values': [16, 18, 24, 32, 64, 96, 128]  
        },
        'hidden_size': {
            'values': [32, 64, 96, 128, 144]  
        },        
        'attention_heads': {
            'values': [1, 2, 3, 4, 5, 6] 
        },
        'embedding_size': {
            'values': [16, 18, 24, 32, 48] 
        },
        'factor': {
          'values': [0.75, 0.8, 0.85, 0.9]
        },
        'sequence_length': {
            'values': [39, 48, 78, 96, 168]   
        },
        'lr': {
            'values': [0.00005, 0.0001, 0.001, 0.01, 0.1] 
        },
        'weight_decay': {
            'values': [0, 0.000001, 0.0001, 0.01, 0.1]  
        },
        'dropout_rate': {
            'values': [0, 0.05, 0.1 , 0.15]  
        },
        'n_encoder_layers': {
            'values': [1, 2, 3, 4, 5]
        },
        'n_decoder_layers': {
            'values': [1, 2, 3, 4, 5] 
        },
        'days_training_length': {
            'values': [31, 62, 124, 248, 365, 520]
        },
    }
}


def set_hyp_config(modelname):
    sweep_config['name'] = modelname
    if modelname == "LSTM":
        print('speciel setting for: ', modelname)
        sweep_config['parameters']['batch_size']['values'] = [1] #LSTM tager kun batch_size på 1
        del sweep_config['parameters']['encoder_length'] #Fjerner alle transformers parametre som ikke er med i LSTM
        del sweep_config['parameters']['attention_heads']
        del sweep_config['parameters']['encoding_size']
        del sweep_config['parameters']['n_encoder_layers']
        del sweep_config['parameters']['n_decoder_layers']
        sweep_config['parameters']['optimizer']['values'] = ['rAdam', 'adam', 'sgd']
        #Tænker der skal være noget til optim her, vil gætte på: sweep_config['parameters']['optimizer']['values'] = ['rAdam', 'adam']
 
    if modelname == "Queryselector" or modelname == "TFT":
        print('speciel setting for: ', modelname)
        #del sweep_config['parameters']['sequence_length']
        #Tænker der skal være noget til optim her, vil gætte på: sweep_config['parameters']['optimizer']['values'] = ['sgd', 'ranger']
 
  
def sweep():
    wandb.init(config=sweep_config)
    run(wandb.config)
 

def wandb_initialize(modelname):
    sweep_config = {
    'name': 'navn',
    'method': 'random', #grid, random, bayesian
    'metric': {
    'name': 'Total_Average_MAE_Loss',
    'goal': 'minimize'  
        },
    'parameters': {
        'batch_size': {
            'values': [16, 18, 24, 32, 64, 96, 128]  
        },
        'hidden_size': {
            'values': [32, 64, 96, 128, 144]  
        },        
        'attention_heads': {
            'values': [1, 2, 3, 4, 5, 6] 
        },
        'embedding_size': {
            'values': [16, 18, 24, 32, 48] 
        },
        'factor': {
          'values': [0.75, 0.8, 0.85, 0.9]
        },
        'sequence_length': {
            'values': [39, 48, 78, 96, 168]   
        },
        'lr': {
            'values': [0.00005, 0.0001, 0.001, 0.01, 0.1] 
        },
        'weight_decay': {
            'values': [0, 0.000001, 0.0001, 0.01, 0.1]  
        },
        'dropout_rate': {
            'values': [0, 0.05, 0.1 , 0.15]  
        },
        'n_encoder_layers': {
            'values': [1, 2, 3, 4, 5]
        },
        'n_decoder_layers': {
            'values': [1, 2, 3, 4, 5] 
        },
        'days_training_length': {
            'values': [31, 62, 124, 248, 365, 520]
        },
    }
}
    #sweep_id = wandb.sweep(sweep_config, project="Yggdrasil", entity="ygg_monkeys") #todo: dette laver en ny sweep.
    sweep_ids = {
        "LSTM":"ywympjpq", 
        "Queryselector":"69vvdc1l", 
        "TFT":"29snzczn"
    }
    wandb.agent(sweep_id=sweep_ids[modelname], function=sweep, project="Yggdrasil", entity="ygg_monkeys")
 

def run(hyper_dick): #Emulate run here

    print("Initializing hyperparameters")
    hyperparameters = {
      "data": "training",
      "seq_len": hyper_dick['sequence_length'], 
      "pred_len": 39,
      "dec_seq_len": hyper_dick['sequence_length'],
      "hidden_size": hyper_dick['hidden_size'],
      "heads": hyper_dick['attention_heads'],
      "n_encoder_layers": hyper_dick['n_encoder_layers'], 
      "encoder_attention": "query_selector_"+ str(hyper_dick['factor']),
      "n_decoder_layers": hyper_dick['n_decoder_layers'],
      "decoder_attention": "full",
      "batch_size": hyper_dick['batch_size'],
      "embedding_size": hyper_dick['embedding_size'],
      "prediction_type": "multiuni", 
      "dropout": hyper_dick['dropout_rate'],
      "fp16": True,
      "deepspeed": True,
      "iterations": 10000,
      "exps": 1,
      "debug": False
    }
    more_hyperparameters = {
      "train_micro_batch_size_per_gpu": 5,
      "gradient_accumulation_steps": 1,
      "steps_per_print": 10,
      "optimizer": {
        "type": "Adam",
        "params": { 
          "lr": hyper_dick['lr'], 
          "weight_decay": hyper_dick['weight_decay']
        }
      },
      "zero_optimization": {
        "stage": 2,
        "allgather_partitions": False,
        "cpu_offload": False
      },
      "fp16": {
        "enabled": True,
        "loss_scale_window": 1000
      }
    }
    with open("settings/hyperparameters.json", "w") as outfile:
        json.dump(hyperparameters, outfile, indent=2)
    with open("settings/ds_config.json", "w") as outfile:
        json.dump(more_hyperparameters, outfile, indent=2)
    print("Done Configuring hyperparameters")



    print("Configure data")
    dates = ["2021-01-10 23:00:00", "2021-04-11 23:00:00", "2021-07-11 23:00:00", "2021-10-10 23:00:00" ]
    for i in range(len(dates)):
      df_season = get_data(dates[i], hyper_dick['days_training_length'], dataset_dropNA)
      split_dataframes(df_season, dates[i]) #Split each season into 7
    read_subset = pd.read_csv("data/training 1 2021-01-10 23:00:00.csv") #First episode and season as starter
    read_subset.to_csv('data/training.csv',index=False)
    print("Done Configuring data")



    print("Starting run")
    !python3 run-ds.py #can be commented out to just serve test-data to setup wand
    get_results()
    print("ending run")
    #mae, rmse, predictions, truth = train(df_season, hyper_dick)


modelnames = ["LSTM", "Queryselector", "TFT"]
modelname = modelnames[1]
set_hyp_config(modelname)
wandb_initialize(modelname)
